In [749]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

In [750]:
import numpy as np
from matplotlib import pyplot as plt
import os
import math
import cv2
import imghdr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import load_model

In [ ]:
# Preprocess data

In [ ]:
data_loc = 'tumor_data'

In [ ]:
dir_list = os.listdir(data_loc)

In [ ]:
extension_list = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
for image_file in dir_list:
        for image in os.listdir(os.path.join(data_loc, image_file)):
            image_path = os.path.join (data_loc, image_file, image)
            try:
                img = cv2.imread(image_path)
                extension = imghdr.what(image_path)
                if extension not in extension_list:
                    os.remove(image_path)
            except Exception as e:
                print("Image issue")

In [ ]:
# Set the data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_loc, labels='inferred')

Found 2870 files belonging to 4 classes.


In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
# corresponding class to tumor type
# Class 0 is giloma tumor
# Class 1 is meningioma tumor
# Class 2 is no tumor
# Class 3 is pituitary tumor


In [ ]:
# Scale data

In [ ]:
data = data.map(lambda x, y: (x/255, y))

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
# Split data into training set, validation set, and testig set

In [ ]:
train_size = 60;
validate_size = 20;
test_size = 10;

In [ ]:
train = data.take(train_size)
validation = data.skip(train_size).take(validate_size)
test = data.skip(train_size + validate_size).take(test_size)

In [ ]:
# Develop the Convolutional Neural Network

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (256,256,3)))
model.add(MaxPooling2D())

In [ ]:
model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

In [ ]:
model.add(Conv2D(64, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

In [ ]:
model.add(Conv2D(128, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

In [ ]:
model.add(Dropout(0.2))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(units=128, activation = 'relu'))
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(4, activation='softmax'))

In [775]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [798]:
# Create logs for training history

In [ ]:
logdir = 'logs'
tensor_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [778]:
history = model.fit(train, epochs=10, validation_data=validation, callbacks=[tensor_callback])

In [820]:
# View model history
history.history

{'loss': [1.0702842473983765,
  0.7365957498550415,
  0.5654796957969666,
  0.4507959187030792,
  0.3400541841983795,
  0.24461619555950165,
  0.18073759973049164,
  0.15900681912899017,
  0.11764902621507645,
  0.09617701917886734],
 'accuracy': [0.5375000238418579,
  0.6963541507720947,
  0.7807291746139526,
  0.828125,
  0.8682291507720947,
  0.9098958373069763,
  0.9354166388511658,
  0.9401041865348816,
  0.9552083611488342,
  0.9651041626930237],
 'val_loss': [0.7109113931655884,
  0.6050018072128296,
  0.5777837038040161,
  0.46538838744163513,
  0.32954996824264526,
  0.3093368411064148,
  0.2838912010192871,
  0.4219532907009125,
  0.27743151783943176,
  0.3331875801086426],
 'val_accuracy': [0.676562488079071,
  0.7484375238418579,
  0.746874988079071,
  0.7984374761581421,
  0.8765624761581421,
  0.887499988079071,
  0.9203125238418579,
  0.864062488079071,
  0.9140625,
  0.910937488079071]}

In [821]:
# Save the model

In [816]:
model.save(os.path.join('models', 'tumorclassificationv2.h5'))

In [822]:
# Load the model

In [815]:
new_model = load_model(os.path.join('models', 'tumorclassification.h5'))

In [824]:
# Create a testing loop to test all the images from my testing folder

In [823]:
def testing_loop ():
    test_loc = "Testing"
    file_name = "glioma_tumor"
    
    for image in os.listdir(os.path.join(test_loc, file_name)):
        image_path = os.path.join (test_loc, file_name, image)
        try:
            test_img = cv2.imread(image_path)
            resize_test = tf.image.resize(test_img, (256, 256))
            plt.imshow(resize_test.numpy().astype(int))
            plt.show()
            prediction =  np.argmax(new_model.predict(np.expand_dims(resize_test/255,0)))
            if prediction == 0:
                 print("This is a glioma tumor")
            elif prediction == 1:
                 print("This is a meningioma tumor")
            elif prediction == 2:
                 print("There is no tumor")
            elif prediction == 3:
                 print("This is a pituitary tumor") 
        except Exception as e:
            print("There was an image error")
                
    

In [ ]:
testing_loop()